In [1]:
#dependencies
import pandas as pd
import datetime as dt
import requests 
import time
import os
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
#start with list of tables
url="https://nuforc.org/webreports/ndxloc.html"
#tables to skip:
skip_list=['INTERNATIONAL',
           'ALBERTA',
           'BRITISH COLUMBIA',
           'MANITOBA',
           'NEW BRUNSWICK',
           'NEWFOUNDLAND',
           'NEWFOUNDLAND AND LABRADOR',
           'NORTHWEST TERRITORY',
           'NOVA SCOTIA',
           'ONTARIO',
           'PRINCE EDW ISLAND',
           'PROV OF QUE',
           'SASKATCHEWAN',
           'YUKON']
#blank master df
master_df=pd.DataFrame(columns=['Date / Time',
                                'City','State',
                                'Shape','Duration',
                                'Summary',
                                'Link to Event'])
master_df['Date / Time']=pd.to_datetime(master_df['Date / Time'],errors='coerce')

In [3]:
#load browser
browser = Browser('chrome', executable_path=ChromeDriverManager().install())

In [4]:
browser.visit(url)

In [5]:
#start souping

html = browser.html
_soup= soup(html, 'html.parser')

In [6]:
#drill down to list of state tables
table=_soup.find('tbody')
#list of tables
links=table.find_all('a')
print(links)

[<a href="ndxlAL.html">ALABAMA</a>, <a href="ndxlAK.html">ALASKA</a>, <a href="ndxlAZ.html">ARIZONA</a>, <a href="ndxlAR.html">ARKANSAS</a>, <a href="ndxlCA.html">CALIFORNIA</a>, <a href="ndxlCO.html">COLORADO</a>, <a href="ndxlCT.html">CONNECTICUT</a>, <a href="ndxlDE.html">DELAWARE</a>, <a href="ndxlDC.html">DISTRICT OF COLUMBIA</a>, <a href="ndxlFL.html">FLORIDA</a>, <a href="ndxlGA.html">GEORGIA</a>, <a href="ndxlHI.html">HAWAII</a>, <a href="ndxlID.html">IDAHO</a>, <a href="ndxlIL.html">ILLINOIS</a>, <a href="ndxlIN.html">INDIANA</a>, <a href="ndxlIA.html">IOWA</a>, <a href="ndxlKS.html">KANSAS</a>, <a href="ndxlKY.html">KENTUCKY</a>, <a href="ndxlLA.html">LOUISIANA</a>, <a href="ndxlME.html">MAINE</a>, <a href="ndxlMD.html">MARYLAND</a>, <a href="ndxlMA.html">MASSACHUSETTS</a>, <a href="ndxlMI.html">MICHIGAN</a>, <a href="ndxlMN.html">MINNESOTA</a>, <a href="ndxlMS.html">MISSISSIPPI</a>, <a href="ndxlMO.html">MISSOURI</a>, <a href="ndxlMT.html">MONTANA</a>, <a href="ndxlNE.html">

In [26]:
#Debug Cell
demo_df=pd.DataFrame(columns=['Date / Time','City','State','Shape','Duration','Summary','Link to Event'])
demo_df['Date / Time']=pd.to_datetime(demo_df['Date / Time'],errors='coerce')
page=links[0].get('href')
state=links[0].text
table_url=f'https://nuforc.org/webreports/{page}'
df=pd.read_html(table_url)[0]
clean_df=df.drop(columns=['Country','Posted','Images'])
#clean_df.head()
browser.visit(table_url)
tab_html = browser.html
tab_soup= soup(tab_html, 'html.parser')
tab_tab=tab_soup.find('tbody')
tab_link=tab_tab.find_all('a')
link_list=[]
for link in tab_link:
    href=link.get('href')
    href_url=f'https://nuforc.org/webreports/{href}'
    link_list.append(href_url)
clean_df['Link to Event']=link_list
clean_df['Date / Time']=pd.to_datetime(clean_df['Date / Time'],errors='coerce')
filter_df=clean_df.loc[clean_df['Date / Time']>="2013-1-1"]
filter_df
demo_df=pd.concat([demo_df,filter_df])
demo_df.head()

#df.to_csv(f"output_csv/{state}.csv")

,Date / Time,City,State,Shape,Duration,Summary,Link to Event
0,2023-05-14 21:55:00,Deer Park,AL,Circle,About 30 mins,Flashing circle/ fire colored/ torpedo,https://nuforc.org/webreports/reports/175/S175...
1,2023-05-14 20:40:00,Athens,AL,Circle,About 2 seconds,Huge yellow-white sphere 5 times the size of t...,https://nuforc.org/webreports/reports/175/S175...
2,2023-05-13 21:30:00,Pike Road,AL,Circle,Hours,Tatyana Bakieva,https://nuforc.org/webreports/reports/175/S175...
3,2023-05-02 04:46:00,Falkville,AL,Light,About 10 minutes,Lights that traveled across the sky faster tha...,https://nuforc.org/webreports/reports/175/S175...
4,2023-05-01 03:32:00,Grand Bay,AL,Sphere,Five to ten minutes,Was looking at stars in the Northern sky and s...,https://nuforc.org/webreports/reports/175/S175...


In [27]:
#Debug Cell 2
demo_df.dtypes

Date / Time      datetime64[ns]
City                     object
State                    object
Shape                    object
Duration                 object
Summary                  object
Link to Event            object
dtype: object

In [9]:
#loop through the links on the page
for link in links:
    #get relevant info and construct url
    page=link.get('href')
    state=link.text
    table_url=f'https://nuforc.org/webreports/{page}'\
    #check to make sure table is not international
    if state in skip_list:
        #if international, just print that the program is skipping it
        print(f"Skipping {state}.")
        time.sleep(.1) 
    else:
        #inform user that program is processing a given state
        print(f"Working on {state} table.")
        #read in info with pandas read_html
        df=pd.read_html(table_url)[0]
        #drop unnecessary columns
        print(f"dropping columns")
        clean_df=df.drop(columns=['Country','Posted','Images'])
        #obtain links to events
        print(f"getting links")
        #direct browser and drill down through html
        browser.visit(table_url)
        tab_html = browser.html
        tab_soup= soup(tab_html, 'html.parser')
        tab_tab=tab_soup.find('tbody')
        #scrape list of html links
        tab_link=tab_tab.find_all('a')
        #generate blank list for holding urls
        print(f"Processing hrefs")
        url_list=[]
        for link in tab_link:
            href=link.get('href')
            href_url=f'https://nuforc.org/webreports/{href}'
            url_list.append(href_url)
        #append the url list as a new column to df
        clean_df['Link to Event']=url_list
        #convert Date/Time to Datetime dtype
        print(f"processing Date/Time")
        clean_df['Date / Time']=pd.to_datetime(clean_df['Date / Time'],errors='coerce')
        #filter info down to last 10 years
        filter_df=clean_df.loc[clean_df['Date / Time']>="2013-1-1"]
        #output csv
        print(f"Exporting {state}.csv")
        filter_df.to_csv(f"output_csv/{state}.csv")
        #concatenate data to master csv
        print(f"Concatenating {state} info to Master csv")
        master_df=pd.concat([master_df,filter_df])
        time.sleep(.1) 
    
    





Working on ALABAMA table.
dropping columns
getting links
Processing hrefs
processing Date/Time
Exporting ALABAMA.csv
Concatenating ALABAMA info to Master csv
Working on ALASKA table.
dropping columns
getting links
Processing hrefs
processing Date/Time
Exporting ALASKA.csv
Concatenating ALASKA info to Master csv
Working on ARIZONA table.
dropping columns
getting links
Processing hrefs
processing Date/Time
Exporting ARIZONA.csv
Concatenating ARIZONA info to Master csv
Working on ARKANSAS table.
dropping columns
getting links
Processing hrefs
processing Date/Time
Exporting ARKANSAS.csv
Concatenating ARKANSAS info to Master csv
Working on CALIFORNIA table.
dropping columns
getting links
Processing hrefs
processing Date/Time
Exporting CALIFORNIA.csv
Concatenating CALIFORNIA info to Master csv
Working on COLORADO table.
dropping columns
getting links
Processing hrefs
processing Date/Time
Exporting COLORADO.csv
Concatenating COLORADO info to Master csv
Working on CONNECTICUT table.
dropping c

In [10]:
#export master csv
master_df.to_csv(f"output_csv/USA.csv")
        

In [11]:
browser.quit

<bound method BaseWebDriver.quit of <splinter.driver.webdriver.chrome.WebDriver object at 0x000002909E26AD60>>